# This notebook contains analysis that provides suggestion to a company on which business will be more profitable

# We are importing the required libraries in the below code

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


# Getting the co-ordinate of Infosys, Mysuru

In [6]:
address = 'Infosys,Mysuru,IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mysuru are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mysuru are 12.3579213, 76.6003325.


# Defining the Foursquare credentials and version

In [7]:
CLIENT_ID = 'BKWLW33L0U51VA3A0X30T2WJWH1KI2N1Z2F2SYPTPYDC3JCX' # your Foursquare ID
CLIENT_SECRET = 'R3PET3QD10NISTU2XD22CCEXYOIEIN4I5Q5YALTCS2EZOAVW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BKWLW33L0U51VA3A0X30T2WJWH1KI2N1Z2F2SYPTPYDC3JCX
CLIENT_SECRET:R3PET3QD10NISTU2XD22CCEXYOIEIN4I5Q5YALTCS2EZOAVW


# Defining the URL to fetch the top 100 venues with in 10 KM of Infosys, Mysuru

In [8]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    10000, 
    100)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=BKWLW33L0U51VA3A0X30T2WJWH1KI2N1Z2F2SYPTPYDC3JCX&client_secret=R3PET3QD10NISTU2XD22CCEXYOIEIN4I5Q5YALTCS2EZOAVW&v=20180605&ll=12.3579213,76.6003325&radius=10000&limit=100'

# Sending the request and examining the results

In [9]:
results = requests.get(url).json()


# Getting the category type of the venue

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Cleaning the Json and storing in dataframe

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Multiplex @ Infosys,Multiplex,12.362263,76.595645
1,Purple Haze Mysore,Pub,12.338032,76.616730
2,Grand Maurya,Indian Restaurant,12.334633,76.595562
3,Mezzaluna Continental,Steakhouse,12.326323,76.619188
4,Downtown,Burger Joint,12.324016,76.625927
5,Corner House,Dessert Shop,12.322813,76.634861
6,Art Trattorias,Restaurant,12.332033,76.628544
7,Santosha,Breakfast Spot,12.334127,76.630237
8,Bowling Alley,Bowling Alley,12.361956,76.596164
9,Grand Mercure Mysuru,Hotel,12.330390,76.645211


# Plotting the near by venues on map

In [12]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

map= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  

map

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

NameError: name 'map_manhattan' is not defined

# Grouping and selecting the categorywise total places in the locality

In [14]:

grouped = nearby_venues.groupby('categories').count().reset_index()
grouped.sort_values('name', ascending=False)

,categories,name,lat,lng
27,Indian Restaurant,11,11,11
10,Café,10,10,10
24,Hotel,8,8,8
33,Pizza Place,5,5,5
39,Shopping Mall,5,5,5
7,Breakfast Spot,4,4,4
36,Restaurant,4,4,4
3,Asian Restaurant,3,3,3
11,Chinese Restaurant,3,3,3
16,Fast Food Restaurant,2,2,2


From the above code it is clear that restaurants are huge in number in the selected location. Shopping malls is 5th in the list. Hence compared to both Resort with outdoor activity proves to be better option